----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery layer - RDP Stream - IPA FX Cross
This notebook demonstrates how to use an RDP stream to subscribe to an FX Cross Intrument Pricing Analytic.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

## Set the configuration file location
For a better ease of use, you have the option to set initialization parameters of the Refinitiv Data Library in the _refinitiv-data.config.json_ configuration file. This file must be located beside your notebook, in your user folder or in a folder defined by the _RD_LIB_CONFIG_PATH_ environment variable. The _RD_LIB_CONFIG_PATH_ environment variable is the option used by this series of examples. The following code sets this environment variable.      

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

## Some Imports to start with

In [4]:
import refinitiv.data as rd
from refinitiv.data.delivery import rdp_stream
import datetime

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [5]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7fcc6753de20 {name='workspace'}>

## Retrieve data

### Define a calback to display received events

In [6]:
def display_event(event, eventType):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(event)

### Create an RDP stream and register event callbacks

In [7]:
stream = rdp_stream.Definition(
    service = None,
    api = "streaming.quantitative-analytics.endpoints.financial-contracts",
    universe = {
        "instrumentType": "FxCross", 
        "instrumentDefinition": {
            "instrumentTag": "USDAUD", 
            "fxCrossType": "FxSpot", 
            "fxCrossCode": "USDAUD"
        }
    }, 
    parameters = None,
    extended_params = {
        "marketData": {
            "fxSpots": [
                {
                    "spotDefinition": {
                        "fxCrossCode": "AUDUSD", 
                        "Source": "Composite"
                    }
                }
            ]
        }
    },
    view = ["InstrumentTag", "FxSpot_BidMidAsk", "ErrorCode", "Ccy1SpotDate", "Ccy2SpotDate"],
).get_stream()

In [8]:
stream.on_response(lambda event, stream: display_event(event, 'response'))
stream.on_update(lambda event, stream: display_event(event, 'update'))
stream.on_alarm(lambda event, stream: display_event(event, 'alarm'))

### Open the stream
This example uses the open() synchronous call to open the stream. Because it is a synchronous call, the first notification (either via on_refresh(), on_status() or on_error()) happens before the open() method returns. If the open_async() asynchronous method is used instead, the first notification happens after open_async() returns.

In [9]:
stream.open()

----------------------------------------------------------

<OpenState.Opened: 'Opened'>


>>> response event received at 15:57:48.058436
{'data': [['USDAUD', {'bid': 1.4799467219180111, 'ask': 1.4803849000740192, 'mid': 1.4801658109960152, 'processingInformation': ''}, '', '2022-12-14', '2022-12-14']], 'type': 'Response', 'streamID': '5'}
----------------------------------------------------------
>>> update event received at 15:57:48.962743
{'data': [['USDAUD', {'bid': 1.4799467219180111, 'ask': 1.4803849000740192, 'mid': 1.4801658109960152, 'processingInformation': ''}, '', '2022-12-14', '2022-12-14']], 'type': 'Update', 'streamID': '5'}
----------------------------------------------------------
>>> update event received at 15:57:50.019615
{'data': [['USDAUD', {'bid': 1.4797277300976621, 'ask': 1.4806040864672787, 'mid': 1.4801659082824705, 'processingInformation': ''}, '', '2022-12-14', '2022-12-14']], 'type': 'Update', 'streamID': '5'}
----------------------------------------------------------
>>> update event received at 15:57:50.968141
{'data': [['USDAUD', {'bid': 1.4

### Close the stream

In [10]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [11]:
rd.close_session()